# 텐서플로우 튜토리얼 #02
# 합성곱 신경망

by [Magnus Erik Hvass Pedersen](http://www.hvass-labs.org/)
/ [GitHub](https://github.com/Hvass-Labs/TensorFlow-Tutorials) / [Videos on YouTube](https://www.youtube.com/playlist?list=PL9Hr9sNUjfsmEu1ZniY0XpHSzl5uihcXZ)

## 소개

이전의 자습서에서는 단순한 선형 모델이 MNIST 데이터 집합에서 손으로 쓴 숫자를 인식하기 위한 분류 정확도가 약 91%라는 것을 보여주었다.

이번 튜토리얼에서는 제안된 연습문제를 만들면 분류 정확도가 약 99% 이상인 간단한 텐서플로우(Convolutional Neural Network in TensorFlow)를 구현할 것이라고 말했다.

합성곱 네트워크는 입력 이미지 전반에 걸쳐 작은 필터를 이동시켜 작동한다. 이는 필터들이 전체 입력 이미지에서 패턴을 인식하기 위해 다시 사용됨을 의미한다. 이것은 같은 수의 변수를 가진 완전 연결 네트워크보다 컨볼루션 네트워크를 훨씬 더 강력하게 만든다. 이것은 차례로 콘볼루션 네트워크를 더 빨리 훈련하도록 만든다.

기본 선형대수, 파이톤과 주피터 공책 편집자에 익숙해져야 한다. TensorFlow의 초보자도 이 튜토리얼로 진행하기 전에 첫 번째 튜토리얼을 공부하고 싶어할 수 있다.

## 플로우차트

다음 도표는 아래에 구현된 합성곱 신경망에서 데이터가 어떻게 흐르는지 대략적으로 보여준다.

![Flowchart](images/02_network_flowchart.png)

입력 이미지는 필터-가중치를 사용하여 첫 번째 경련층에서 처리된다. 이로 인해 16개의 새로운 이미지가 생성되며, 각 필터마다 한 개의 합성 층이 생성된다. 영상도 하향 샘플링되므로 영상 해상도가 28x28에서 14x14로 감소한다.

이 16개의 작은 이미지들은 두 번째 합성곱층에서 처리된다. 우리는 이 16개 채널 각각에 대한 필터-가중치가 필요하고, 이 계층의 각 출력 채널에 대한 필터-가중치가 필요하다. 출력 채널은 36개가 있으므로 두 번째 경련층에는 총 16 x 36 = 576개의 필터가 있다. 결과 영상은 7x7 픽셀로 다시 다운샘플링된다.

제2합성곱층의 출력은 각각 7x7픽셀의 36개 이미지다. 그리고 나서 이것들은 길이 7 x 7 x 36 = 1764의 단일 벡터로 평평하게 되는데, 이것은 128개의 뉴런(또는 원소)으로 완전히 연결된 층에 대한 입력으로 사용된다. 이것은 10개의 뉴런으로 완전히 연결된 또 다른 층으로 공급되는데, 각 계층마다 하나씩, 즉 영상의 등급을 결정하는 데 사용되는, 즉 영상에 어떤 숫자가 묘사되어 있는가 하는 것이다.

합성곱형 필터는 처음에 무작위로 선택되기 때문에 분류는 무작위로 이루어진다. 입력 영상의 예측 클래스와 실제 클래스 사이의 오차는 이른바 교차 엔트로피로 측정된다. 그런 다음 최적기는 분화의 체인 규칙을 사용하여 자동으로 이 오류를 콘볼루션 네트워크를 통해 다시 전파하고 분류 오류를 개선하도록 필터 가중치를 업데이트한다. 이는 분류 오류가 충분히 낮을 때까지 반복적으로 수천 번 이루어진다.

이러한 특정 필터 무게와 중간 이미지는 하나의 최적화 실행의 결과물이며 이 노트북을 다시 실행하면 다르게 보일 수 있다.

TensorFlow의 계산은 실제로 단일 영상 대신 영상 배치에서 수행되므로 계산의 효율성이 향상된다는 점에 유의하십시오. 이는 실제로 TensorFlow에서 구현했을 때 플로우차트가 한 가지 더 데이터 경감을 갖는다는 것을 의미한다.

## 합성곱층

다음 도표는 첫 번째 경련 층에서 이미지를 처리하는 기본 아이디어를 보여준다. 입력 이미지는 숫자 7과 4개의 이미지 복사본이 여기에 표시되므로 필터가 이미지의 다른 위치로 어떻게 이동되고 있는지 보다 명확하게 알 수 있다. 필터의 각 위치에 대해 필터와 필터 아래의 영상 픽셀 사이에 도트 제품이 계산되고 있으며, 출력 영상에 하나의 픽셀이 생성된다. 따라서 필터를 전체 입력 이미지에서 이동하면 새 이미지가 생성된다.

빨간색 필터-중량은 필터가 입력 영상의 검은 픽셀에 대해 양성 반응을 보인다는 것을 의미하고, 파란 픽셀은 필터가 검은 픽셀에 대해 음성 반응을 보인다는 것을 의미한다.
이 경우 필터는 출력 영상에서 해당 라인에 대한 강한 반응에서 볼 수 있듯이 7자리의 수평선을 인식하는 것으로 나타난다.
      

![Convolution example](images/02_convolution.png)

입력을 가로질러 필터를 이동시키기 위한 스텝 사이즈를 스트레이드라고 한다. 필터를 수평(x축)으로 이동하기 위한 보폭과 수직으로 이동하기 위한 보폭(y축)이 있다.

아래 소스 코드에서 보폭은 양방향으로 1로 설정되는데, 이는 필터가 입력 이미지의 왼쪽 상단 모서리에서 시작하여 각 단계에서 1픽셀 오른쪽으로 이동되고 있다는 것을 의미한다. 필터가 영상의 끝에 도달하면 필터가 다시 왼쪽으로 이동하고 영상 아래로 1픽셀 이동한다. 이 작업은 필터가 입력 이미지의 오른쪽 하단 모서리에 도달하고 전체 출력 이미지가 생성될 때까지 계속된다.

필터가 입력 이미지의 하단은 물론 우측 끝단에 도달하면 0(흰색 픽셀)으로 패딩할 수 있다. 이로 인해 출력 이미지가 입력 이미지와 정확히 같은 차원이 된다.

더 나아가서, 콘볼루션의 출력은 음수 값이 0으로 설정되어 있기 때문에 출력만 양수하도록 하는 소위 정류된 선형 단위(ReLU)를 통해 전달될 수도 있다. 출력은 또한 2x2 픽셀의 작은 창을 고려하고 그 중 가장 큰 픽셀만 유지하는 이른바 max-pooling에 의해 하향 샘플링될 수 있다. 이는 입력 이미지의 분해능을 28x28 픽셀에서 14x14 픽셀까지 반반으로 한다.

두 번째 경련층은 16개의 입력 채널을 사용하기 때문에 더욱 복잡하다는 점에 유의하십시오. 우리는 각 입력 채널마다 별도의 필터를 원하기 때문에 우리는 하나의 필터 대신에 16개의 필터가 필요하다. 더욱이 우리는 두 번째 경련층으로부터 36개의 출력 채널을 원하기 때문에, 두 번째 경련층을 위해서는 총 16 x 36 = 576 필터가 필요하다. 이것이 어떻게 작동하는지 이해하는 것은 좀 어려울 수 있다.

## 텐서플로우 2

이 튜토리얼은 2016년 텐서플로 v.1을 이용해 개발했다. TensorFlow v.2에서는 상당한 API 변경이 있었다. 본 자습서는 \"v.1 호환성 모드\"에서 TF2를 사용하며, TensorFlow의 작동 방식을 배우는 데 여전히 유용하지만 TF2에서는 약간 다르게 구현해야 한다(Keras API의 자습서 03C 참조). 구글의 엔지니어들이 텐서플로 API를 업데이트할 때마다 이런 튜토리얼을 계속 업데이트하는 것은 나에게 너무 큰 일이기 때문에 결국 이 튜토리얼이 작동을 멈출 수도 있다.
    

## Imports

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math

In [2]:
# 이 오래된 v.1 코드와 함께 텐서플로 v.2를 사용하십시오.
# 예: TF2.\n에서 자리 표시자 변수 및 세션이 변경됨
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

ModuleNotFoundError: No module named 'tensorflow'

이것은 Python 3.6 (Anaconda)과 TensorFlow 버전을 사용하여 개발되었다.

In [ ]:
tf.__version__

## 신경망 구성

합성곱 신경망의 구성은 편의를 위해 여기에 정의되어 있으므로 이러한 숫자를 쉽게 찾아 변경하고 노트북을 재실행할 수 있다.

In [ ]:
# 합성곱 신경망 1.
filter_size1 = 5          # 컨볼루션 필터는 5 x 5 픽셀입니다.
num_filters1 = 16         # 이 필터들은 16개가 있습니다.

# 합성곱 신경망 2.,
filter_size2 = 5          # 컨볼루션 필터는 5 x 5 픽셀입니다.
num_filters2 = 36         # 이 필터들은 36개가 있습니다.

# 완전히 연결된 층.
fc_size = 128             # 완전히 연결된 층에 있는 뉴런의 수입니다.

## 데이터 로드

MNIST 데이터 세트는 약 12MB로 주어진 경로에 위치하지 않으면 자동으로 다운로드된다.

In [ ]:
from mnist import MNIST
data = MNIST(data_dir="data/MNIST/")

MNIST 데이터 세트는 현재 로드되었으며 이미지용 70.000개의 영상과 클래스 번호로 구성되어 있다. 데이터 세트는 3개의 상호 배타적인 하위 세트로 분할된다. 이번 튜토리얼에서는 훈련과 시험 세트만 사용할 것이라고 말했다.

In [ ]:
print("Size of:")
print("- Training-set:\t\t{}".format(data.num_train))
print("- Validation-set:\t{}".format(data.num_val))
print("- Test-set:\t\t{}".format(data.num_test))

편리를 위해 데이터 치수 일부를 복사하십시오.

In [ ]:
# 이미지의 각 차원에 있는 픽셀 수입니다.
img_size = data.img_size

# 영상은 이 길이의 1차원 배열로 저장된다.
img_size_flat = data.img_size_flat

# 배열을 재구성하는 데 사용되는 이미지의 높이와 폭을 가진 튜플.,
img_shape = data.img_shape

# 클래스 수, 열 자리마다 한 클래스씩.
num_classes = data.num_classes

# 영상의 색상 채널 수: 그레이 스케일의 경우 1 채널.
num_channels = data.num_channels

### 이미지 플롯을 위한 도우미 함수

9개의 영상을 3x3 그리드에 플로팅하고 각 이미지 아래에 참 클래스와 예측 클래스를 작성하는 데 사용되는 함수

In [3]:
def plot_images(images, cls_true, cls_pred=None):
    assert len(images) == len(cls_true) == 9
    
    # 3x3 서브플롯으로 피규어를 만든다.
    fig, axes = plt.subplots(3, 3)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        # 플롯 이미지.
        ax.imshow(images[i].reshape(img_shape), cmap='binary')

        # 진실되고 예측된 클래스를 보여라.
        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true[i])
        else:
            xlabel = "True: {0}, Pred: {1}".format(cls_true[i], cls_pred[i])

        # x축에 클래스를 라벨로 표시한다.
        ax.set_xlabel(xlabel)
        
        # 줄거리에서 눈금을 제거하라.
        ax.set_xticks([])
        ax.set_yticks([])
    
    # 여러 플롯을 사용하여 플롯이 올바르게 표시되도록 하십시오.
    # 노트북 한 셀 안에.
    plt.show()

### 데이터가 올바른지 보기 위해 몇 개의 이미지를 플롯하십시오.

In [4]:
# 시험 세트에서 첫 번째 영상을 얻으십시오.
images = data.x_test[0:9]

# 그 이미지들에 대한 진정한 수업을 받으세요.
cls_true = data.y_test_cls[0:9]

# 위의 도우미 기능을 사용하여 이미지와 라벨을 플롯하십시오.
plot_images(images=images, cls_true=cls_true)

NameError: name 'data' is not defined

## 텐서플로우 그래프

텐서플로우의 전체 목적은 파이톤에서 직접 동일한 계산을 수행할 경우보다 훨씬 효율적으로 실행할 수 있는 이른바 계산 그래프를 갖추는 것이다. 텐서플로우는 실행해야 하는 전체 연산 그래프를 알고 있는 반면, NumPy는 한 번에 하나의 수학 연산만을 알고 있기 때문에 텐서플로우는 NumPy보다 효율적일 수 있다.

TensorFlow는 모델의 성능을 향상시키기 위해 그래프의 변수를 최적화하는 데 필요한 그라데이션도 자동으로 계산할 수 있다. 이는 그래프가 단순한 수학 식을 조합한 것이어서 전체 그래프의 그라데이션은 파생상품의 체인룰을 이용해 계산할 수 있기 때문이다.

TensorFlow는 GPU뿐만 아니라 멀티 코어 CPU도 활용할 수 있으며 구글은 TPU(Tensor Processing Units)라고 불리며 GPU보다 더 빠른 텐서플로우만을 위한 특수 칩까지 만들었다.

TensorFlow 그래프는 아래에 자세히 설명될 다음과 같은 부분으로 구성된다:

* 그래프로 데이터를 입력하는 데 사용되는 자리 표시자 변수.
* 컨볼루션 네트워크를 더 잘 수행할 수 있도록 최적화될 변수.
* 수녀원 네트워크의 수학 공식.
* 변수의 최적화를 안내하는 데 사용할 수 있는 비용 측정값.
* 변수를 업데이트하는 최적화 방법.

또한 TensorFlow 그래프는 본 자습서에서 다루지 않는 TensorBoard를 사용하여 표시되는 데이터 로깅과 같은 다양한 디버깅 문도 포함할 수 있다.

## 새 변수 생성에 대한 도우미 함수

특정 셰이프에 새 텐서플로 변수를 만들고 랜덤 값으로 초기화하는 기능. 이 시점에서 초기화가 실제로 수행되는 것이 아니라 텐서플로 그래프에서 정의되고 있을 뿐이라는 점에 유의하십시오.

In [5]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

In [6]:
def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

###새로운 콘볼루션 레이어 작성을 위한 도우미 기능

이 함수는 텐서플로우 계산 그래프에 새로운 경련층을 만든다. 여기서는 실제로 계산된 것이 없고, 단지 수학 공식을 텐서플로 그래프에 추가하는 것뿐입니다.

입력이 다음과 같은 치수를 갖는 4차원 텐서인 것으로 가정한다:

1. 이미지 번호.
2. 각 영상의 Y축.
3. 각 영상의 X축.
4. 각 이미지의 채널.

입력 채널이 컬러 채널이거나, 입력이 이전의 경련 층에서 생성된 경우 필터 채널일 수 있다는 점에 유의하십시오.

출력은 다음과 같은 치수를 가진 또 다른 4차원 텐서이다:

1. 입력과 같은 이미지 번호.
2. 각 영상의 Y축. 2x2 풀링을 사용할 경우 입력 영상의 높이와 너비를 2.으로 나눈다.
3. 각 영상의 X축. 디토.
4. 경련 필터에 의해 생성된 채널 

In [7]:
def new_conv_layer(input,              # 이 전 층
                   num_input_channels, # 수. 이전 층의 채널 
                   filter_size,        # 각 필터의 넓이와 높이
                   num_filters,        # 필터 수
                   use_pooling=True):  # 2X2 맥스 풀링을 사용하여라

    # 경련용 필터-체중의 모양.
    # 이 형식은 TensorFlow API.에 의해 결정된다.
    shape = [filter_size, filter_size, num_input_channels, num_filters]

    # 주어진 형상으로 새로운 역기 aka. 필터를 만들어라.
    weights = new_weights(shape=shape)

    # 필터마다 하나씩 새로운 biases을 만들어라.
    biases = new_biases(length=num_filters)

    # 합성을 위한 TensorFlow작전을 만든다.
    # 모든 차원에서 장족의 진보는 1로 설정됨을 유의한다.
    # 첫 걸음과 마지막 걸음은 항상 1이어야 한다
    # 첫 번째는 이미지 번호와 n을 위한 것이기 때문에
    # 마지막은 입력 채널용입니다.
    # but expect=[1, 2, 1, 1]은 필터을 의미할 것이다.
    #는 영상의 x축과 y축에 걸쳐 2픽셀 이동한다.
    # 패딩은 입력 이미지을 의미하는 'SAME'으로 설정되어 있으며
    #은 0으로 패딩되어 출력 크기가 동일하다.
    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')

    # 합성곱의 결과에 편견을 더하라.
    # 각 필터 채널에 바이어스 값이 추가된다.
    layer += biases

    # 이미지 해상도를 낮추기 위해 풀링을 사용하시겠습니까?
    if use_pooling:
        # 이건 우리가 의미한 2x2 최대 풀링이다.
        # 2x2 창을 고려하고 가장 큰 값을 선택하십시오.
        #윈도우마다. 그럼 2픽셀을 다음 윈도우로 옮기자.
        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')

    # 수정 선형 단위(ReLU).
    # 입력 픽셀 x에 대한 최대값(x, 0)을 계산한다.
    # 이것은 공식에 비선형성을 더하고 우리를 허용한다.
    # 더 복잡한 기능을 배우기 위해
    layer = tf.nn.relu(layer)

    # reLU는 보통 풀링 전에 실행된다는 점에 유의하십시오.
    # but since relu(max_pool(x)) == max_pool(relu(x)) 
    # 릴루-트랜스포트의 75%를 먼저 최대값으로 절약한다.

    # 결과 도면층과 필터-가중치을 모두 반환한다.
    # 나중에 추를 그려야 하니까,
    return layer, weights

### 도면층을 평평하게 하는 도우미 함수

콘볼루션 레이어는 4차원의 출력 텐서를 생성한다. 콘볼루션 레이어 이후에 완전연결 레이어를 추가할 예정이기 때문에 완전연결 레이어에 대한 입력으로 사용할 수 있는 4차원 텐서(them tensor)를 2차원(them)으로 줄여야 한다.

In [8]:
def flatten_layer(layer):
    # 입력층의 모양을 잡아라.
    layer_shape = layer.get_shape()

The shape of the input layer is assumed to be:
layer_shape == [num_images, img_height, img_width, num_channels]

    # 특징의 수는: img_hight * img_width * num_channels이다.
    # 텐서플로우(TensorFlow)의 함수를 사용해 이를 계산할 수 있다.
    num_features = layer_shape[1:4].num_elements()
    
    # 레이어를 [num_images, num_features]로 재구성한다.
    # 두번째 차원의 크기를 정했다는 것을 주목하라.
    # num_features 에 적용하고 첫 번째 차원의 크기를 -1로
    # which means the size in that dimension is calculated\n",
    # 치수의 크기가 계산됨을 의미함
    layer_flat = tf.reshape(layer, [-1, num_features])

    # 이제 평평해진 층의 모양이다:
    # [num_images, img_height * img_width * num_channels]

    # 평평해진 층과 형상의 수를 모두 반환한다.
    return layer_flat, num_features

SyntaxError: invalid syntax (<ipython-input-8-0c334b031f4c>, line 5)

### 완전히 연결된 새 레이어를 만들기 위한 도우미 함수

이 함수는 텐서플로용 계산 그래프에서 완전히 연결된 새로운 레이어를 생성한다. 여기서는 실제로 계산된 것이 없고, 단지 수학 공식을 텐서플로 그래프에 추가하는 것뿐입니다.

입력이 `[num_images, num_inputs]` 2차원 텐서인 것으로 추정된다. 출력물은 2차원 텐서 형태의 `[num_images, num_outputs]`이다.

In [9]:
def new_fc_layer(input,          # 이전 층
                 num_inputs,     # 수, 이전 층으로으부터의 입력
                 num_outputs,    # 수, 출력
                 use_relu=True): # ReLU를 사용하여라

    # 새로운 weights와 biases을 만들어라.
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    # 도면층을 매트릭스 곱하기으로 계산한다.
    # 입력과 가중치를 입력한 다음 바이어스-바이어를 추가한다.
    layer = tf.matmul(input, weights) + biases

    # ReLU 사용해라?\n",
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

## 자리 표시자 변수

플레이스홀더 변수는 우리가 그래프를 실행할 때마다 변경될 수 있는 텐서플로 계산 그래프의 입력 역할을 한다. 우리는 이것을 자리 표시자 변수 공급이라고 부르는데, 그것은 아래에 더 자세히 설명되어 있다.

먼저 우리는 입력 영상에 대한 자리 표시자 변수를 정의한다. 이를 통해 TensorFlow 그래프에 입력되는 영상을 변경할 수 있다. 이것은 이른바 텐서(tensor)로, 단지 다차원 벡터(tensor) 데이터 타입은 `float32`로, 형태는 `[none, img_size_flat]`로 설정되어 있는데, 여기서 `None`은 각각의 이미지가 길이 `img_size_flat`의 벡터인 상태에서 임의의 수의 영상을 담을 수 있다는 것을 의미한다.

In [10]:
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')

NameError: name 'tf' is not defined

합성층에서는 `x`가 4차원 텐서로 인코딩될 것으로 예상하므로 그 모양이 `[num_images, img_hight, img_width, num_channels]`가 되도록 재구성해야 한다고 말했다. `img_hight == img_width==img_size`와 `num_images`는 첫 번째 치수 크기에 대해 -1을 사용하여 자동으로 추론할 수 있다. 따라서 재편성 작업은 다음과 같다.

In [11]:
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])

NameError: name 'tf' is not defined

이어 다음에는 자리 표시자 변수 `x`에 입력된 이미지와 관련된 실제 레이블에 대한 자리 표시자 변수가 있다. 이 자리 표시자 변수의 모양은 임의의 수의 레이블을 포함할 수 있다는 뜻의 `[None, num_classes]`이며, 각 레이블은 길이가 10인 `num_classes`의 벡터다.

In [12]:
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

NameError: name 'tf' is not defined

클래스 번호에 대한 자리 표시자 변수도 가질 수 있지만 대신 argmax를 사용해 계산하겠다. 이 연산자는 텐서플로우 연산자이므로 이 시점에서는 아무것도 계산되지 않는다는 점에 유의하십시오.

In [13]:
y_true_cls = tf.argmax(y_true, axis=1)

NameError: name 'tf' is not defined

### 합성층 1

첫 번째 합성곱층을 만들어라. 입력으로 `x_image`를 사용하고 각각 `filter_size1`과 같은 폭과 높이를 갖는 `num_filter1`의 서로 다른 필터를 만든다. 마지막으로 2x2 max-pooling을 사용하여 이미지를 절반 크기로 다운샘플링하고자 한다. 

In [14]:
layer_conv1, weights_conv1 = \
    new_conv_layer(input=x_image,
                   num_input_channels=num_channels,
                   filter_size=filter_size1,
                   num_filters=num_filters1,
                   use_pooling=True)

NameError: name 'x_image' is not defined

합성층이 출력할 텐서의 모양을 확인하라. (?, 14, 14, 16) 임의의 수의 이미지(이것은 ?)가 있고, 각 이미지는 너비가 14픽셀이고 높이가 14픽셀이며, 필터마다 하나의 채널인 16개의 다른 채널이 있다는 것을 의미한다."

In [15]:
layer_conv1

NameError: name 'layer_conv1' is not defined

\합성층이 출력할 텐서의 모양을 확인하라. (?, 14, 14, 16) 임의의 수의 이미지(이것은 ?)가 있고, 각 이미지는 너비가 14픽셀이고 높이가 14픽셀이며, 필터마다 하나의 채널인 16개의 다른 채널이 있다는 것을 의미한다."


In [16]:
layer_conv2, weights_conv2 = \
    new_conv_layer(input=layer_conv1,
                   num_input_channels=num_filters1,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   use_pooling=True)

NameError: name 'layer_conv1' is not defined

이 콘볼루션 층에서 출력될 텐서의 모양을 확인해 보십시오. 모양은 (?, 7, 7, 36)인데, 다시 ?는 임의의 수의 영상이 있음을 의미하며, 각 이미지는 폭과 높이가 7픽셀이고, 36개의 채널이 필터마다 하나씩 있다."


In [17]:
layer_conv2

NameError: name 'layer_conv2' is not defined

### 플래튼 레이어

합성층은 4차원 텐터를 출력한다. 이제 우리는 이것들을 완전히 연결된 네트워크의 입력으로 사용하기를 희망하고 있는데, 이 입력은 텐서를 2차원 텐서로 재구성하거나 평평하게 만들어야 한다.

In [18]:
layer_flat, num_features = flatten_layer(layer_conv2)

NameError: name 'flatten_layer' is not defined

텐더에 모양(?, 1764)이 있는지 확인해 보라면서 각각 길이 1764의 벡터로 납작해진 임의의 이미지 수가 있다고 말했다. 1764 = 7 x 7 x 36에 유의하십시오.

In [19]:
layer_flat

NameError: name 'layer_flat' is not defined

In [20]:
num_features

NameError: name 'num_features' is not defined

### 완전히 연결된 층 1

완전히 연결된 계층을 네트워크에 추가하십시오. 그 입력은 이전의 콘볼루션으로부터 평평해진 층이다. 완전히 연결된 층에 있는 뉴런이나 노드의 수는 fc_size이다. 리루(ReLU)가 이용되기 때문에 비선형 관계를 배울 수 있다.

In [21]:
layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=fc_size,
                         use_relu=True)

NameError: name 'layer_flat' is not defined

완전히 연결된 층의 출력이 임의의 수의 영상이 있고 'fc_size' ==128인 모양(?, 128)의 텐서인지 확인한다.

In [22]:
layer_fc1

NameError: name 'layer_fc1' is not defined

### 완전히 연결된 층 2

입력 이미지가 속하는 클래스 10개 중 어떤 클래스를 결정할 수 있도록 길이 10의 벡터를 출력하는 또 다른 완전 연결 레이어를 추가하십시오. 이 레이어에는 ReLU가 사용되지 않는다는 점에 유의하십시오.

In [23]:
layer_fc2 = new_fc_layer(input=layer_fc1,
                         num_inputs=fc_size,
                         num_outputs=num_classes,
                         use_relu=False)

NameError: name 'layer_fc1' is not defined

In [24]:
layer_fc2

NameError: name 'layer_fc2' is not defined

### 예측된 클래스

두 번째 완전 연결 레이어는 입력 이미지가 10개 등급 각각에 속할 가능성이 얼마나 되는지 추정한다. 그러나 이러한 추정치는 숫자가 매우 작거나 클 수 있기 때문에 다소 거칠고 해석하기 어려운 것이므로, 각 원소가 0과 1 사이에 제한되고 10 원소가 합하여 1로 제한되도록 정상화하고자 한다. 이를 소프트맥스(softmax) 기능으로 계산해 그 결과를 `y_pred`에 저장한다.

In [25]:
y_pred = tf.nn.softmax(layer_fc2)

NameError: name 'tf' is not defined

클래스 넘버는 가장 큰 원소의 지수다.

In [26]:
y_pred_cls = tf.argmax(y_pred, axis=1)

NameError: name 'tf' is not defined

### 최적화할 비용 기능

모형이 입력 영상을 더 잘 분류하도록 하려면 어떻게든 모든 네트워크 계층에 대한 변수를 변경해야 한다. 이를 위해서는 먼저 모델 `y_pred`의 예측 출력 `y_true`를 원하는 출력 `y_true`와 비교하여 모델이 현재 얼마나 잘 작동하는지 알아야 한다.

크로스 엔트로피는 분류에 사용되는 성능 측정값이다. 교차-엔트로피는 항상 양의 함수로서 모델의 예측 출력이 원하는 출력과 정확히 일치하면 교차-엔트로피는 0이다. 따라서 최적화의 목표는 교차 엔트로피를 최소화하여 네트워크 계층의 변수를 변경하여 가능한 한 0에 가깝게 하는 것이다.

텐서플로우는 교차 엔트로피를 계산하는 기능이 내장되어 있다. 함수는 내부적으로 소프트맥스를 계산하기 때문에 이미 소프트맥스가 적용된 `y_pred`가 아닌 `layer_fc2`의 출력을 직접 사용해야 한다는 점에 유의한다."

In [27]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,
                                                        labels=y_true)

NameError: name 'tf' is not defined

우리는 이제 각 이미지 분류에 대한 교차 엔트로피를 계산했기 때문에 모델이 각 이미지에서 개별적으로 얼마나 잘 수행되는지 측정할 수 있다. 그러나 교차 엔트로피를 사용하여 모델의 변수의 최적화를 유도하려면 단일 스칼라 값이 필요하므로 모든 이미지 분류에 대한 교차 엔트로피의 평균을 단순히 취한다.

In [28]:
cost = tf.reduce_mean(cross_entropy)

NameError: name 'tf' is not defined

### 최적화된 매서드

이제 최소화해야 할 비용측정이 생겼으니, 그러면 최적기를 만들 수 있다. 이 경우 그라데이션 강하의 고급 형태인 `애덤 옵티마이저(Adam Optimizer)`이다.

이 시점에서는 최적화가 수행되지 않는다는 점에 유의하십시오. 사실 아무것도 계산되지 않고, 나중에 실행할 수 있도록 최적화 도구-개체를 텐서플로 그래프에 추가하기만 하면 된다.

In [29]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

NameError: name 'tf' is not defined

### 수행 측정

사용자에게 진행 상황을 보여주기 위해 몇 가지 성과 조치가 더 필요하다.

예측된 클래스가 각 이미지의 참된 클래스와 동일한지 여부를 나타내는 술래들의 벡터다. 

In [30]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)

NameError: name 'tf' is not defined

이것은 우선 술집의 벡터를 물에 뜨게 타이핑하여 분류 정확도를 계산하여 거짓이 0이 되고 참이 1이 되게 한 다음 이 숫자들의 평균을 계산한다.

In [31]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

NameError: name 'tf' is not defined

## 텐서플로우 수행

### 텐서플로우 섹션 만들기

TensorFlow 그래프가 생성되면 그래프를 실행하는 데 사용되는 TensorFlow 세션을 생성해야 한다. 

In [32]:
session = tf.Session()

NameError: name 'tf' is not defined

### 변수 초기화

`weights`와 `biases`는 최적화를 시작하기 전에 초기화해야 한다.

In [33]:
session.run(tf.global_variables_initializer())

NameError: name 'session' is not defined

## 최적화 반복을 수행하는 도우미 함수

훈련 세트에는 5만 5천 개의 이미지가 있다. 이 모든 영상을 사용하여 모델의 그라데이션 계산에 오랜 시간이 걸린다. 따라서 최적기의 각 반복에는 작은 이미지 배치만 사용한다.

RAM이 부족하여 컴퓨터가 고장 나거나 속도가 매우 느려지면 이 숫자를 줄이려고 할 수 있지만, 최적화 반복을 더 많이 수행해야 할 수도 있다.

In [34]:
train_batch_size = 64

네트워크 계층의 변수를 점진적으로 개선할 수 있도록 여러 번의 최적화 반복을 수행하는 기능. 각 반복에서 교육 세트에서 새로운 데이터 배치를 선택한 다음 텐서플로우는 해당 교육 샘플을 사용하여 최적기를 실행한다. 진행상황은 100회마다 인쇄된다.

In [35]:
# 지금까지 수행된 총 반복 횟수에 대한 카운터.
total_iterations = 0

def optimize(num_iterations):
    # 로컬 복사본이 아닌 글로벌 변수를 업데이트하십시오.
    global total_iterations

    # 아래 시간 사용량 인쇄에 사용되는 시작 시간.
    start_time = time.time()

    for i in range(total_iterations,
                   total_iterations + num_iterations):

        # 훈련 사례를 한 묶음으로 받아라.
        # x_batch는 이제 한 묶음의 이미지를 가지고 있다. 그리고
        # y_true_messages는 그러한 영상의 진정한 라벨이다.
        x_batch, y_true_batch, _ = data.random_batch(batch_size=train_batch_size)

        # 적절한 이름을 가진 명령어에 배치
        # 텐서플로우 그래프에서 자리 표시자 변수의 경우.
        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch}

        # 이 교육 데이터 배치를 사용하여 최적화 도구를 실행한다.
        # TensorFlow는 feed_dict_train에 변수를 할당한다.
        # 자리 표시자 변수에 연결한 다음 최적화 도구를 실행하십시오.
        session.run(optimizer, feed_dict=feed_dict_train)

        # 100회마다 상태 출력,
        if i % 100 == 0:
            # 트레이닝 세트의 정확도를 계산한다.
            acc = session.run(accuracy, feed_dict=feed_dict_train)

            # 인쇄 메시지
            msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}"

            # 출력해랴.
            print(msg.format(i + 1, acc))

    # 수행된 전체 반복 횟수를 업데이트하십시오.
    total_iterations += num_iterations

    # 종료 시간.
    end_time = time.time()

    # 시작 시간과 종료 시간의 차이.
    time_dif = end_time - start_time

    # 사용 시간 출력.
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))

## 예시 오류를 플롯하는 도우미 함수

잘못 분류된 테스트 세트의 이미지 예시를 플로팅하는 함수.

In [36]:
def plot_example_errors(cls_pred, correct):
    # This function is called from print_test_accuracy() below.

    # cls_pred is an array of the predicted class-number for
    # all images in the test-set.

    # correct is a boolean array whether the predicted class
    # is equal to the true class for each image in the test-set.

    # Negate the boolean array.
    incorrect = (correct == False)
    
    # Get the images from the test-set that have been
    # incorrectly classified.
    images = data.x_test[incorrect]
    
    # Get the predicted classes for those images.
    cls_pred = cls_pred[incorrect]

    # Get the true classes for those images.
    cls_true = data.y_test_cls[incorrect]
    
    # Plot the first 9 images.
    plot_images(images=images[0:9],
                cls_true=cls_true[0:9],
                cls_pred=cls_pred[0:9])

### 혼동 행렬을 표시하는 도우미 함수

In [37]:
def plot_confusion_matrix(cls_pred):
     # 아래 print_test_정확도()에서 부르는 말이다.

    # cls_pred는 예측된 클래스 번호의 배열이다.
    # 테스트 세트의 모든 이미지.

    # 테스트 세트에 대한 진정한 분류를 얻으십시오.
    cls_true = data.y_test_cls
    
    # sklearn을 이용하여 혼란 매트릭스를 얻는다.
    cm = confusion_matrix(y_true=cls_true,
                          y_pred=cls_pred)

    # 혼란 매트릭스를 텍스트로 출력한다.
    print(cm)

    # 혼동 행렬을 이미지로 표시.
    plt.matshow(cm)

    # 플롯을 다양하게 조정.
    plt.colorbar()
    tick_marks = np.arange(num_classes)
    plt.xticks(tick_marks, range(num_classes))
    plt.yticks(tick_marks, range(num_classes))
    plt.xlabel('Predicted')
    plt.ylabel('True')

    # 여러 플롯을 사용하여 플롯이 올바르게 표시되도록 하십시오.
    # 단일 노트북 셀에.
    plt.show()

##성능을 보여주는 도우미 함수

시험 세트에 분류 정확도를 인쇄하는 기능.

테스트셋의 모든 영상에 대한 분류를 계산하는 데 시간이 걸리므로, 이 함수에서 위의 함수를 직접 호출하여 결과를 다시 사용하는 것이므로 각 함수에 의해 분류를 다시 계산할 필요가 없다.

이 함수는 컴퓨터 메모리를 많이 사용할 수 있으므로 테스트 세트가 더 작은 배치로 분할되는 것에 유의하십시오. 컴퓨터에 RAM이 거의 없는데 작동이 안 되면 배치 크기를 줄이려고 하면 된다.

In [38]:
# 테스트 세트를 이 크기의 작은 배치로 분할하십시오.
test_batch_size = 256

def print_test_accuracy(show_example_errors=False,
                        show_confusion_matrix=False):

    # 테스트 세트의 영상 수입니다.
    num_test = data.num_test

    # 예측 클래스에 어레이를 할당하십시오.
    # 일괄적으로 계산하여 이 배열로 채울 것이다.
    cls_pred = np.zeros(shape=num_test, dtype=np.int)

    # 이제 배치에 대한 예측 클래스를 계산하십시오.
    # 우리는 모든 배치들을 통해 반복할 것이다.
    # 좀 더 영리하고 피토닉한 방법이 있을 것이다.

    # 다음 배치의 시작 지수는 i으로 표시된다.
    i = 0

    while i < num_test:
        # 다음 배치에 대한 끝 지수는 j로 표시된다..
        j = min(i + test_batch_size, num_test)

        # 인덱스 i와 j 사이의 테스트 세트에서 이미지 가져오기.
        images = data.x_test[i:j, :]

        # 연결된 레이블 가져오기.
        labels = data.y_test[i:j, :]

        # 이러한 이미지 및 레이블로 피드 딕트 만들기.
        feed_dict = {x: images,
                     y_true: labels}

        # 텐서플로우를 사용하여 예측 클래스 계산.
        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)

        # 다음 배치에 대한 시작 인덱스를 다음으로 설정
        # 현재 배치에 대한 엔드 인덱스.
        i = j

    # 테스트 세트의 실제 클래스 번호에 대한 편의 변수.
    cls_true = data.y_test_cls

    # 각 이미지가 올바르게 분류되었는지 여부를 나타내는 부울 배열 만들기.
    correct = (cls_true == cls_pred)

    # 올바르게 분류된 영상 수 계산.
    # 부울 배열을 합할 때 False는 0을 의미하고 True는 1을 의미한다..
    correct_sum = correct.sum()

    # 분류 정확도는 정확하게 분류된 수입니다.
    # 영상을 테스트 세트의 총 영상 수로 나눈다..
    acc = float(correct_sum) / num_test

    # 정확도 출력.
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))

    # 필요한 경우 일부 잘못된 분류 예제 그림 그리기
    if show_example_errors:
        print("Example errors:")
        plot_example_errors(cls_pred=cls_pred, correct=correct)

    #원하는 경우 혼동 행렬을 표시하십시오.
    if show_confusion_matrix:
        print("Confusion Matrix:")
        plot_confusion_matrix(cls_pred=cls_pred)

## 최적화 전 성능

모델 변수가 초기화되었을 뿐 전혀 최적화되지 않았기 때문에 테스트 세트의 정확도가 매우 낮기 때문에 영상을 무작위로 분류할 뿐이다.  

In [39]:
print_test_accuracy()

NameError: name 'data' is not defined

## 최적화 1번 반복 후 성능

최적기에 대한 학습 비율이 매우 낮게 설정되어 있기 때문에 분류 정확도는 한 번의 최적화 반복만으로 크게 개선되지 않는다.  

In [40]:
optimize(num_iterations=1)

NameError: name 'data' is not defined

In [41]:
print_test_accuracy()

NameError: name 'data' is not defined

## 최적화를 100회 반복한 후의 성능

100회의 최적화 반복을 거쳐, 그 모델은 분류 정확도를 현저히 향상시켰다.

In [42]:
optimize(num_iterations=99) # 우리는 이미 위에서 1회 반복을 했다.

NameError: name 'data' is not defined

In [43]:
print_test_accuracy(show_example_errors=True)

NameError: name 'data' is not defined

## 1000번의 최적화 반복 후 성능

1000회 최적화 반복 후 이 모델은 시험 세트의 정확도를 90% 이상으로 크게 높였다.

In [44]:
optimize(num_iterations=900) # We performed 100 iterations above.

NameError: name 'data' is not defined

In [45]:
print_test_accuracy(show_example_errors=True)

NameError: name 'data' is not defined

## 최적화 반복 10,000회 이후 성능

최적화 1만 번 반복한 후, 이 모델은 약 99%의 시험 세트에 대한 분류 정확도를 갖는다. 

In [46]:
optimize(num_iterations=9000) # 1000번 반복 수행

NameError: name 'data' is not defined

In [47]:
print_test_accuracy(show_example_errors=True,
                    show_confusion_matrix=True)

NameError: name 'data' is not defined

## 가중치 및 층의 시각화

왜 합성 신경 네트워크가 손으로 쓴 숫자를 인식할 수 있는지 이해하기 위해 우리는 이제 합성 필터의 무게와 결과 출력 이미지를 시각화할 것이다. 

##합성 가중치 플로팅을 위한 도우미 함수

In [48]:
def plot_conv_weights(weights, input_channel=0):
    # 4-dim 변수\n에 대한 텐서플로우 ops라고 가정한다.
    # 예: weights_sv1 또는 weights_sv2.

    # TensorFlow에서 중량 변수 값 검색.
    # 아무것도 계산되지 않기 때문에 피드 딕트가 필요하지 않다..
    w = session.run(weights)

    # 가중치에 대한 가장 낮은 값과 가장 높은 값 가져오기.
    # 이것은 전체에서 색 강도를 보정하는 데 사용된다.
    # 서로 비교될 수 있도록 하는 이미지들.
    w_min = np.min(w)
    w_max = np.max(w)

    # Conv. 도면층에 사용된 필터 수.
    num_filters = w.shape[3]

     # 플롯할 그리드 수.
    # 필터 수의 반올림, 제곱근.
    num_grids = math.ceil(math.sqrt(num_filters))
    
    # 하위 그림 그리드를 사용하여 그림 작성.
    fig, axes = plt.subplots(num_grids, num_grids)

    # 모든 필터-가중치 그림 그리기.
    for i, ax in enumerate(axes.flat):
         # 유효한 필터-가중치만 표시.
        if i<num_filters:
    # 입력 채널의 i번째 필터에 대한 가중치 가져오기.
    # 형식에 대한 자세한 내용은 new_conv_layer()를 참조하십시오.
    # 이 4차원 텐서의.
            img = w[:, :, input_channel, i]

            # 플롯 이미지.
            ax.imshow(img, vmin=w_min, vmax=w_max,
                      interpolation='nearest', cmap='seismic')
        
        # 그림에서 눈금 제거.
        ax.set_xticks([])
        ax.set_yticks([])
    
    # 그림이 여러 그림으로 올바르게 표시되는지 확인
    # 단일 노트북 셀에.
    plt.show()

### 컨벌루션 층의 출력을 플로팅하기 위한 도우미 함수

In [49]:
def plot_conv_layer(layer, image):
    # 레이어가 4-dim 텐서를 출력하는 텐서플로우 op이라고 가정하십시오.
    # 컨벌루션 층의 결과물이지,
    # 예: layer_layer1 또는 layer_layer2.

    # 하나의 이미지만 포함하는 피드 딕트 생성.
    # y_true는 y_true이기 때문에 공급할 필요가 없다는 점에 유의하십시오.
    # 이 계산에 사용되지 않는.
    feed_dict = {x: [image]}

     # 레이어의 출력 값 계산 및 검색
     # 그 이미지를 입력할 때.
    values = session.run(layer, feed_dict=feed_dict)

    # Conv. 도면층에 사용된 필터 수.
    num_filters = values.shape[3]

      # 플롯할 그리드 수.
      # 필터 수의 반올림, 제곱근.
    num_grids = math.ceil(math.sqrt(num_filters))
    
    # 하위 그림 그리드를 사용하여 그림 작성.
    fig, axes = plt.subplots(num_grids, num_grids)

        # 모든 필터의 출력 영상 플롯.
    for i, ax in enumerate(axes.flat):
        # 유효한 필터에 대한 영상만 표시.
        if i<num_filters:
            # i번째 필터 사용의 출력 이미지 가져오기.
            # 형식에 대한 자세한 내용은 new_conv_layer()를 참조하십시오.
            # 이 4차원 텐서의.
            img = values[0, :, :, i]

            # 플롯 이미지.
            ax.imshow(img, interpolation='nearest', cmap='binary')
        
        # 그림에서 눈금 제거.
        ax.set_xticks([])
        ax.set_yticks([])
    
      # 그림이 여러 그림으로 올바르게 표시되는지 확인
    # 단일 노트북 셀에.
    plt.show()

### 입력 이미지

이미지를 플롯하는 데 도움이 되는 기능.

In [50]:
def plot_image(image):
    plt.imshow(image.reshape(img_shape),
               interpolation='nearest',
               cmap='binary')

    plt.show()

아래 예제로 사용할 테스트 세트의 이미지를 플롯하십시오.

In [51]:
image1 = data.x_test[0]
plot_image(image1)

NameError: name 'data' is not defined

테스트 세트에서 다른 예제 이미지를 플롯하십시오.

In [52]:
image2 = data.x_test[13]
plot_image(image2)

NameError: name 'data' is not defined

### 컨벌루션 층 1

이제 첫 번째 합성 층에 대한 필터-가중치를 플롯하십시오.

양가중치는 빨간색이고 음가중치는 파란색이라는 점에 유의하십시오.  

In [53]:
plot_conv_weights(weights=weights_conv1)

NameError: name 'weights_conv1' is not defined

이러한 각 경련 필터를 첫 번째 입력 영상에 적용하면 다음과 같은 출력 영상이 제공되며, 이는 두 번째 경련층에 대한 입력으로 사용된다. 이러한 이미지는 원래 입력 이미지의 절반 해상도인 14 x 14 픽셀로 하향 샘플링된다는 점에 유의하십시오.

In [54]:
plot_conv_layer(layer=layer_conv1, image=image1)

NameError: name 'layer_conv1' is not defined

다음 이미지는 두 번째 이미지에 경련 필터를 적용한 결과 입니다.

In [55]:
plot_conv_layer(layer=layer_conv1, image=image2)

NameError: name 'layer_conv1' is not defined

이러한 이미지들로 볼 때 경련 필터의 목적이 무엇일지 알 수 없다. 마치 다른 각도에서 빛이 비추고 이미지에서 그림자를 드리우는 것처럼 입력 이미지의 여러 변형을 만들었을 뿐인 것 같다고 말했다.

### 합성층 2

이제 두 번째 합성층에 대한 필터-가중치를 그려보십시오.

첫 번째 합성층에서 16개의 출력 채널이 있는데, 이것은 두 번째 합성층으로 16개의 입력 채널이 있다는 것을 의미한다. 두 번째 콘블레이어에는 각 입력 채널에 대한 필터-중량 집합이 있다. 첫 번째 채널의 필터-위치를 계획하는 것부터 시작합시다.

다시 한 번 긍정적인 체중은 빨간색이고 부정적인 체중은 파란색이라는 것을 주목하라. 

In [56]:
plot_conv_weights(weights=weights_conv2, input_channel=0)

NameError: name 'weights_conv2' is not defined

제2의 콘볼루션층에는 16개의 입력 채널이 있으므로 이와 같은 필터-중량도를 15개 더 만들 수 있다. 두 번째 채널의 필터-가중치로 한 개만 더 만들면 돼.

In [57]:
plot_conv_weights(weights=weights_conv2, input_channel=1)

NameError: name 'weights_conv2' is not defined

차원이 높기 때문에 이러한 필터가 어떻게 적용되는지 이해하고 추적하기가 어려울 수 있다.,

이러한 콘볼루션 필터를 첫 번째 콘블레이어로부터 uput된 영상에 적용하면 다음과 같은 영상이 나온다.

이러한 것들은 다시 7 x 7 픽셀로 하향 샘플링되는데 이는 첫 번째 콘블레이어 이미지 해상도의 절반이다.

In [58]:
plot_conv_layer(layer=layer_conv2, image=image1)

NameError: name 'layer_conv2' is not defined

그리고 두 번째 이미지에 필터-가중치를 적용한 결과 입니다.

In [59]:
plot_conv_layer(layer=layer_conv2, image=image2)

NameError: name 'layer_conv2' is not defined

이 이미지들에서 두 번째 합성 층은 입력 영상의 선과 패턴을 감지할 수 있을 것으로 보이며, 이는 원래 입력 영상의 국부적 변화에 덜 민감하다.

이어 이러한 영상들은 평평하게 펴져 완전히 연결된 층에 입력되지만, 그것은 여기에 나타나지 않는다고 덧붙였다."

### 텐서플로우 섹션 닫기

이제 텐서플로우를 이용한 작업이 끝났기 때문에 그 자원을 공개하기 위해 세션을 닫는다.

In [60]:
# 수정 및 실험을 원할 경우에 대비하여 언급된 사항.
# 노트북을 다시 시작할 필요 없이.
# session.close()

## 결론

우리는 컨볼루션 신경 네트워크가 자습서 #01의 단순한 선형 모델보다 손으로 쓴 숫자를 인식하는 데 훨씬 더 효과가 있다는 것을 보았다. 콘볼루션 네트워크는 단순한 선형 모델의 경우 91%에 불과했던 것에 비해, 분류 정확도는 약 99% 또는 일부 조정을 할 경우 훨씬 더 높다.

그러나 콘볼루션 네트워크는 구현도 훨씬 복잡하고 필터-가중치을 보면 왜 작동하는지, 왜 실패하는지도 분명하지 않다.

따라서 우리는 콘볼루션 신경망을 보다 쉽게 프로그램할 수 있는 방법과 그들의 내부 작업을 시각화하는 더 나은 방법을 원한다.

## 연습

텐서플로우(TensorFlow)로 여러분의 실력을 향상시키는 데 도움이 될 수 있는 몇 가지 운동 제안사항들이다. 텐서플로우를 제대로 사용하는 방법을 익히기 위해서는 텐서플로우를 직접 체험해 보는 것이 중요하다.

변경하기 전에 이 노트북을 백업하십시오.

* 매개변수를 변경하지 않고 노트북을 여러 번 실행해도 동일한 결과를 얻는가? 무작위의 근원은 무엇인가?
* 10,000회의 최적화 반복 실행. 결과가 더 좋은가?
* 최적기의 학습 속도를 변경하십시오.
* 콘볼루션 필터 수, 필터 크기, 완전히 연결된 층의 뉴런 수 등 레이어의 구성을 변경한다.
* 완전히 연결된 층 뒤에 이른바 드롭아웃 층을 추가한다. 분류 정확도를 계산할 때 드롭아웃 확률은 0이어야 하므로 이 확률에 대한 자리 표시자 변수가 필요하다.
* ReLU와 콘볼루션 층의 최대 풀링 순서를 변경한다. 계산이 같은가? 그것을 계산하는 가장 빠른 방법은 무엇인가? 얼마나 많은 계산이 저장되었는가? 지그모이드 기능과 평균 풀링에도 효과가 있는가?,
* 하나 이상의 콘볼루션 및 완전 연결 레이어를 추가한다. 성능에 도움이 되는가?
* 여전히 좋은 결과를 얻을 수 있는 가장 작은 구성은 무엇인가?
* 완전히 연결된 마지막 레이어에서 ReLU를 사용해 보십시오. 성능이 변하나? 왜?
* 합성층에서는 풀링을 사용하지 마십시오. 분류 정확도와 교육 시간이 달라지는가?
* 콘볼루션에서 최대 풀링 대신 2x2의 보폭을 사용해 보십시오. 뭐가 다른데?
* 이 소스 코드를 너무 많이 보지 말고 직접 프로그램을 리메이크하십시오.
* 그 프로그램이 어떻게 작동하는지 친구에게 설명하라.  

## 라이센스(MIT)

저작권 (c) 2016 by [Magnus Erik Hvass Pedersen](http://www.hvass-labs.org/),

이 소프트웨어의 사용, 복사, 수정, 병합, 게시, 배포, 하위 라이선스 및/또는 판매 권한을 포함하여 제한 없이 본 소프트웨어 및 관련 문서 파일(\"Software\")의 사본을 입수하는 모든 사용자에게 무료로 권한을 부여한다.다음과 같은 조건에 따라 소프트웨어가 제공될 수 있는 이들:

위의 저작권 고지 및 이 허가 고지는 소프트웨어의 모든 사본 또는 상당 부분에 포함되어야 한다.

상품성, 특정 목적에의 적합성 및 비침해의 보증을 포함하되 이에 국한되지 않는 모든 종류의 명시적 또는 묵시적 보증 없이 \"있는 그대로 \" 소프트웨어는 제공된다. 어떠한 경우에도 저작자나 저작권 소유자는 책임을 지지 않는다. 그렇지 않으면 소프트웨어, 소프트웨어 사용 또는 기타 거래에서 발생하거나, 소프트웨어와 연결되거나, 연결된다."
    